In [ ]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=ee037b29e12aa78008b1483762bf1fb1543b0b4014044a07ebdf53434eae48d0
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark import SparkContext,SparkConf
conf=SparkConf().setAppName("Skew").setMaster("local[*]")
sc=SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=Skew>

# Loading data skew

In [ ]:
import numpy as np
import random

key_1=['a']*10
key_2=['b']*600000
key_3=['c']*800
key_4=['d']*10000

keys=key_1 + key_2 + key_3 + key_4

values_1 = list(np.random.randint(low=1,high=100,size=len(key_1)))
values_2 = list(np.random.randint(low=1,high=100,size=len(key_2)))
values_3 = list(np.random.randint(low=1,high=100,size=len(key_3)))
values_4 = list(np.random.randint(low=1,high=100,size=len(key_4)))

values = values_1 + values_2 + values_3 + values_4

pair_skew=list(zip(keys,values))
# pair_skew

In [ ]:
#load the data in RDD
rdd = sc.parallelize(pair_skew,8)

# groupByKey

In [ ]:
data_sample=[(1,4),(2,2),(2,1),(3,5),(2,5),(2,10),(2,7),(3,4),(2,1),(2,4),(4,4)]
#making partitions
rdd_sample = sc.parallelize(data_sample,)
rdd_sample.glom().collect()

[[(1, 4), (2, 2), (2, 1)],
 [(3, 5), (2, 5), (2, 10)],
 [(2, 7), (3, 4), (2, 1), (2, 4), (4, 4)]]

In [ ]:
rdd_sample_grouped = rdd_sample.groupByKey()

for item in rdd_sample_grouped.collect():
    print(item[0],[value for value in item[1]])

#show partitions
rdd_sample_grouped.glom().collect()

3 [5, 4]
1 [4]
4 [4]
2 [2, 1, 5, 10, 7, 1, 4]


[[(3, <pyspark.resultiterable.ResultIterable at 0x7e418079a410>)],
 [(1, <pyspark.resultiterable.ResultIterable at 0x7e4180798910>),
  (4, <pyspark.resultiterable.ResultIterable at 0x7e4180799b70>)],
 [(2, <pyspark.resultiterable.ResultIterable at 0x7e4180799150>)]]

In [ ]:
grouped_rdd = rdd.groupByKey().cache()

#run a simple data transformation (using map()) on the skewed data
grouped_rdd.map(lambda pair : (pair[0],[(i+10) for i in pair[1]])).count()

4

## Mitigate the data skewness using salting

In [ ]:
def salting(val):
  tmp=val+"_"+str(random.randint(0,5))
  return tmp


In [ ]:
#Salting Method
rdd_salting=rdd.map(lambda x: (salting(x[0],x[1])))

#actaul code
grouped_rdd = rdd.groupByKey().cache()

#run a simple data transformation (using map()) on the skewed data
grouped_rdd.map(lambda pair : (pair[0],[(i+10) for i in pair[1]])).count()

4

# Run a shuffle sortByKey() to see how the skew effectd computation resources


In [ ]:
rdd_sort = rdd.sortByKey(ascending=False,numPartitions=4)
rdd_sort.count()

610810

# Mitigating data skewness : SALTING

In [ ]:
# rdd_sort = rdd_salting.sortByKey(ascending=False, numPartitions=4)
# rdd_sort.count()

# Run a shuffle join() to see how the skew effects computation resources

In [ ]:
small_rdd1 = sc.parallelize([(2,3),(1,3),(1,4),(3,1),(5,1)],3)
small_rdd2 = sc.parallelize([(4,3),(0,1),(1,2),(2,1)],2)

print(small_rdd1.collect())
print(small_rdd2.collect())

[(2, 3), (1, 3), (1, 4), (3, 1), (5, 1)]
[(4, 3), (0, 1), (1, 2), (2, 1)]


In [ ]:
join1 = small_rdd1.join(small_rdd2)
join1.collect()

[(1, (3, 2)), (1, (4, 2)), (2, (3, 1))]

In [ ]:
join1.getNumPartitions()

5

In [ ]:
join1.glom().collect()

[[], [(1, (3, 2)), (1, (4, 2))], [(2, (3, 1))], [], []]

In [ ]:
import numpy as np
import random

key_1=['a']*10
key_3=['c']*800
key_4=['d']*10000

keys=key_1 + key_2 + key_4

values_1 = list(np.random.randint(low=1,high=100,size=len(key_1)))
values_2 = list(np.random.randint(low=1,high=100,size=len(key_2)))
values_4 = list(np.random.randint(low=1,high=100,size=len(key_4)))

values = values_1 + values_2 + values_4

pair_data=list(zip(keys,values))

In [ ]:
small_rdd=sc.parallelize(pair_data,2)
small_rdd

ParallelCollectionRDD[73] at readRDDFromFile at PythonRDD.scala:289

In [ ]:
#Join without salting
rdd_join=rdd.join(small_rdd)
rdd_join.map(lambda x:int(x[1][0]+ x[1][1])).reduce(lambda a,b:a+b)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Mitigate data skewness : SALTING

In [ ]:
import random
# add random value to the key --> (key,randint)
rdd_new=rdd.map(lambda x: ((x[0],random.randomint(0,10)),x[1])).cache()

# replicate the small data
small_rdd_new = small_rdd.cartesian(sc.parallelize(range(0,11))).map(lambda x:(x[0][0],x[1],x[0][1])).cache()

In [ ]:
#join with salting

rdd_join = rdd_new.join(small_rdd_new)
# rdd_join.map(lambda x: int(x[1][0] + x[1][1])).reduce(lambda a,b:a+b)